<a href="https://colab.research.google.com/github/Rifat951/Depression-detection-using-Twitter-Dataset/blob/main/AML_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re


In [150]:
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [151]:
import ftfy

In [152]:
np.random.seed(1234)

DEPRES_NROWS = 3200  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 12000 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

In [153]:
df_processd = pd.read_csv("/content/twitter_processed_data.csv", sep = '|', header = None, nrows = DEPRES_NROWS)

##Display data

In [154]:
df_processd.head(5)

,0
0,989292962323615744|2018-04-25|23:59:57|Eastern...
1,989292959844663296|2018-04-25|23:59:56|Eastern...
2,989292951716155392|2018-04-25|23:59:54|Eastern...
3,989292873664393218|2018-04-25|23:59:35|Eastern...
4,989292856119472128|2018-04-25|23:59:31|Eastern...


In [155]:
#naming the column
df_processd.columns = ["Text"]

In [156]:
df_processd.head()

,Text
0,989292962323615744|2018-04-25|23:59:57|Eastern...
1,989292959844663296|2018-04-25|23:59:56|Eastern...
2,989292951716155392|2018-04-25|23:59:54|Eastern...
3,989292873664393218|2018-04-25|23:59:35|Eastern...
4,989292856119472128|2018-04-25|23:59:31|Eastern...


In [157]:
## split the column into sub-columns and visualize the data properly

df_processed_col = df_processd.Text.str.split(pat='|',n = 6, expand=True)
df_processed_col.head(10)

,0,1,2,3,4,5,6
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,"1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,"
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,"1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,"1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
5,989292843125616641,2018-04-25,23:59:28,Eastern Standard Time,PHOTOGPRODIG,my anxiety and my depression fighting over who...,"0|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
6,989291000911642625,2018-04-25,23:52:09,Eastern Standard Time,sp4cecounty,wow she's suddenly cured my depression and gav...,"0|2|9|,,,,,,,,,,,,,,,,,,,,,,,,,"
7,989290997346643968,2018-04-25,23:52:08,Eastern Standard Time,sassi_klassi,"I am officially done with @kanyewest. him, the...","0|0|0|,,,,,,,,,,,,,,,,,,,,,"
8,989290972973600770,2018-04-25,23:52:02,Eastern Standard Time,therealazukaobi,Me: what's wrong?My girl: *looks up at me with...,"0|0|2|#examseason,,,,,,,,,,,,,,,,,,,,,,,,,"
9,989290951012044800,2018-04-25,23:51:57,Eastern Standard Time,KateSusabu,@AusBorderForce @PeterDutton_MP @shanebazzi Ag...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"


In [158]:
df_processed_col.columns = ["Twitter_Id", "Date", "Time", "Time_Zone", "User_Id", "Tweet", "Rating"]

In [159]:
df_processed_col.head()

,Twitter_Id,Date,Time,Time_Zone,User_Id,Tweet,Rating
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,"1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,"
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,"1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,"1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"


In [160]:
df_processed_col.to_csv('processed_csv.csv')

In [161]:
## value.counts() gets us the unique values in our dataset
df_processed_col["Tweet"].value_counts()

Mom's depression tied to kids' emotional, intellectual development  https://ift.tt/2HtVZCE                                                                                                                                                                                            15
1-800-273-8255 - Suicide Hotline877-226-3111 - Addiction Hotline844-228-2962 - Eating Disorder Hotline877-455-0628 - Self Harm Hotline888-640-5174 - Depression HotlineShare this for someone who might really need these phone numbers right now.                                     5
According to a new study ketamine has fast-acting benefits for depression:  http://www.bbc.co.uk/news/health-43753073 …                                                                                                                                                                5
Depression sucks                                                                                                                                             

In [162]:
display_of_unique_val = df_processed_col['Tweet'].value_counts()/df_processed_col.shape[0]*100
print('Shape of Dataset -> ' , df_processed_col.shape)

Shape of Dataset ->  (3200, 7)


In [163]:
display_of_unique_val.to_csv('display_data.csv')

In [177]:
df_new = pd.read_csv("/content/dsiplay_data.csv", header = None)
df_new.columns = ["Text", "Unique_Val"]
df_new.head()

,Text,Unique_Val
0,"Mom's depression tied to kids' emotional, inte...",0.46875
1,1-800-273-8255 - Suicide Hotline877-226-3111 -...,0.15625
2,According to a new study ketamine has fast-act...,0.15625
3,Depression sucks,0.09375
4,Lonely millennials twice as likely to experien...,0.09375


In [182]:
df_disp = df_new["Unique_Val"].value_counts()
df_disp.head()

0.03125    3131
0.06250      19
0.15625       2
0.09375       2
0.46875       1
Name: Unique_Val, dtype: int64

In [191]:
df_disp_ori = df_processed_col["Tweet"].value_counts()
df_disp_ori.to_csv('new_disp.csv')  

In [194]:
df_disp1 = pd.read_csv("/content/new_disp.csv", header = None)
df_disp1.columns = ["Text", "Unique_Val"]    
df_disp1.head()

,Text,Unique_Val
0,"Mom's depression tied to kids' emotional, inte...",15
1,1-800-273-8255 - Suicide Hotline877-226-3111 -...,5
2,According to a new study ketamine has fast-act...,5
3,Depression sucks,3
4,Lonely millennials twice as likely to experien...,3


In [195]:
df_disp1 = df_disp1["Unique_Val"].value_counts()
df_disp1.head()

1     3131
2       19
5        2
3        2
15       1
Name: Unique_Val, dtype: int64

In [196]:
#drop some columns with person info and which are not required to give data more clarity

df_process_red_col = df_processed_col.drop(["Twitter_Id", "Date", "Time", "Time_Zone", "User_Id"], axis = 1)


In [197]:
df_process_red_col.head()

,Tweet,Rating
0,The lack of this understanding is a small but ...,"1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,"
1,i just told my parents about my depression and...,"1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
2,depression is something i don't speak about ev...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
3,Made myself a tortilla filled with pb&j. My de...,"1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"
4,@WorldofOutlaws I am gonna need depression med...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"


In [198]:
#usually in twitter dataset some words are in short form
#we are looking to expand them

# Expand Contraction
wordList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(wordList.keys()))

In [199]:
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer

In [200]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return wordList[match.group(0)]
    return c_re.sub(replace, text)

In [201]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            # remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(
                re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

            # fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)

            # expand contraction
            tweet = expandContractions(tweet)

            # remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            # stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet)
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            # stemming words
            tweet = PorterStemmer().stem(tweet)

            cleaned_tweets.append(tweet)

            print(tweets)

    return cleaned_tweets

In [202]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

df['A'] = df['A'].apply(add_2)
print (df)

# or #

df['A'].transform(add_2)
print (df)

In [203]:
## apply the clean tweets into our dataframe and store it to another value to access it later on
df_preprocessed_data = df_process_red_col.apply(clean_tweets)

Streaming output truncated to the last 5000 lines.
4        0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,
                     ...               
3195    0|0|6|,,,,,,,,,,,,,,,,,,,,,,,,,
3196    0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,
3197    0|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,
3198     1|0|1|,,,,,,,,,,,,,,,,,,,,,,,,
3199      0|0|0|,,,,,,,,,,,,,,,,,,,,,,,
Name: Rating, Length: 3200, dtype: object
0       1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,
1       1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,
2        0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,
3       1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,
4        0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,
                     ...               
3195    0|0|6|,,,,,,,,,,,,,,,,,,,,,,,,,
3196    0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,
3197    0|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,
3198     1|0|1|,,,,,,,,,,,,,,,,,,,,,,,,
3199      0|0|0|,,,,,,,,,,,,,,,,,,,,,,,
Name: Rating, Length: 3200, dtype: object
0       1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,
1       1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,
2        0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,
3       1|0|0|,,,,,,,,,,,

In [204]:
df_preprocessed_data.head(10)

,Tweet,Rating
0,the lack understanding small significant part ...,1 0 3
1,told parents depression hard get gen x people ...,1 0 2
2,depression something speak even going also dou...,0 0 0
3,made tortilla filled pb j my depression cured ...,1 0 0
4,i gon na need depression meds soon rainouts sp...,0 0 0
5,anxiety depression fighting whos day https twi...,0 0 2
6,wow suddenly cured depression gave us world pe...,0 2 9
7,i officially done neptunes justin timberlake t...,0 0 0
8,me wrong my girl looks look depression stress ...,0 0 2
9,mp agreed as someone long experience clinical ...,0 0 0
